In [1]:
import argparse
import time

# Define the Sudoku grid size
grid_size = 9

# Define the number of cells to fill in for the --hint flag
hint_cells = 3

# Define the possible values for each cell
possible_values = set(range(1, grid_size + 1))

# Define a function to validate the input grid
def is_valid(grid):
    # Check that each row contains unique values
    for row in grid:
        if len(set(row)) != grid_size:
            return False
    # Check that each column contains unique values
    for col in range(grid_size):
        if len(set(grid[row][col] for row in range(grid_size))) != grid_size:
            return False
    # Check that each box contains unique values
    box_size = int(grid_size ** 0.5)
    for box_row in range(box_size):
        for box_col in range(box_size):
            values = []
            for row in range(box_row * box_size, (box_row + 1) * box_size):
                for col in range(box_col * box_size, (box_col + 1) * box_size):
                    values.append(grid[row][col])
            if len(set(values)) != grid_size:
                return False
    # The grid is valid
    return True

# Define a function to find the next empty cell in the grid
def find_empty(grid):
    for row in range(grid_size):
        for col in range(grid_size):
            if grid[row][col] == 0:
                return row, col
    # There are no empty cells
    return None, None

# Define a function to solve the Sudoku puzzle using backtracking
def solve_backtracking(grid):
    # Find the next empty cell in the grid
    row, col = find_empty(grid)
    # If there are no empty cells, the puzzle is solved
    if row is None:
        return grid
    # Try each possible value for the empty cell
    for value in possible_values:
        # Check if the value is valid in the current cell
        if value in grid[row] or value in (grid[i][col] for i in range(grid_size)):
            continue
        box_row, box_col = row // 3 * 3, col // 3 * 3
        if value in (
            grid[i][j]
            for i in range(box_row, box_row + 3)
            for j in range(box_col, box_col + 3)
        ):
            continue
        # Assign the value to the current cell
        grid[row][col] = value
        # Recursively solve the remaining puzzle
        solution = solve_backtracking(grid)
        # If the solution is found, return it
        if solution is not None:
            return solution
        # If the solution is not found, backtrack
        grid[row][col] = 0
    # If none of the values work, the puzzle is unsolvable
    return None

In [2]:
# Define a function to solve the Sudoku puzzle using constraint propagation
def solve_propagation(grid):
    # Create a set of sets to represent the possible values for each cell
    possible = [[set(possible_values) for _ in range(grid_size)] for _ in range(grid_size)]
    # Initialize the possible values based on the input grid
    for row in range(grid_size):
        for col in range(grid_size):
            if grid[row][col] != 0:
                possible[row][col] = set()
                possible[row][col].add(grid[row][col])
    # Define a function to propagate the constraints
    def propagate():
        while True:
            # Check if the puzzle is solved
            if all(len(possible[row][col]) == 1 for row in range(grid_size) for col in range(grid_size)):
                return [[next(iter(possible[row][col])) for col in range(grid_size)] for row in range(grid_size)]
            # Check if there is an unsolvable cell
            if any(len(possible[row][col]) == 0 for row in range(grid_size) for col in range(grid_size)):
                return None
            # Propagate the constraints
            for row in range(grid_size):
                for col in range(grid_size):
                    # Skip solved cells
                    if len(possible[row][col]) == 1:
                        continue
                    # Update the possible values based on the row, column, and box constraints
                    row_values = set(possible[row][col] for col in range(grid_size))
                    col_values = set(possible[row][col] for row in range(grid_size))
                    box_row, box_col = row // 3 * 3, col // 3 * 3
                    box_values = set(
                        possible[i][j]
                        for i in range(box_row, box_row + 3)
                        for j in range(box_col, box_col + 3)
                    )
                    possible[row][col] = possible[row][col] & (possible_values - row_values - col_values - box_values)
    # Propagate the constraints to solve the puzzle
    return propagate()

In [4]:
# Define the Sudoku puzzle to solve
puzzle = [
    [0, 5, 9, 0, 0, 0, 0, 0, 2],
    [7, 0, 0, 9, 0, 0, 0, 0, 0],
    [0, 0, 8, 0, 0, 0, 0, 0, 6],
    [3, 0, 0, 0, 0, 0, 0, 4, 0],
    [0, 4, 0, 0, 0, 0, 6, 0, 0],
    [0, 0, 6, 0, 0, 0, 8, 0, 0],
    [0, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 7, 0, 0],
    [0, 0, 0, 2, 0, 0, 0, 0, 0],
]

# Solve the puzzle using backtracking
solution_backtracking = solve_backtracking(puzzle)

# Display the solution
if solution_backtracking is not None:
    print("Solution using backtracking:")
    for row in solution_backtracking:
        print(row)
else:
    print("No solution using backtracking.")

# Solve the puzzle using constraint propagation
solution_propagation = solve_propagation(puzzle)

# Display the solution
if solution_propagation is not None:
    print("Solution using constraint propagation:")
    for row in solution_propagation:
        print(row)
else:
    print("No solution using constraint propagation.")

Solution using backtracking:
[1, 5, 9, 3, 6, 7, 4, 8, 2]
[7, 6, 2, 9, 4, 8, 1, 3, 5]
[4, 3, 8, 1, 2, 5, 9, 7, 6]
[3, 1, 7, 6, 8, 2, 5, 4, 9]
[8, 4, 5, 7, 1, 9, 6, 2, 3]
[2, 9, 6, 4, 5, 3, 8, 1, 7]
[6, 7, 1, 5, 3, 4, 2, 9, 8]
[5, 2, 3, 8, 9, 1, 7, 6, 4]
[9, 8, 4, 2, 7, 6, 3, 5, 1]
Solution using constraint propagation:
[1, 5, 9, 3, 6, 7, 4, 8, 2]
[7, 6, 2, 9, 4, 8, 1, 3, 5]
[4, 3, 8, 1, 2, 5, 9, 7, 6]
[3, 1, 7, 6, 8, 2, 5, 4, 9]
[8, 4, 5, 7, 1, 9, 6, 2, 3]
[2, 9, 6, 4, 5, 3, 8, 1, 7]
[6, 7, 1, 5, 3, 4, 2, 9, 8]
[5, 2, 3, 8, 9, 1, 7, 6, 4]
[9, 8, 4, 2, 7, 6, 3, 5, 1]
